# Mã hoá biến định tính dạng String

> Stata amateur <br>
> Ngày 25/08/2021

Lấy cảm hứng từ hàm `get_dummies()` của `pandas` và `OneHotEncoder()/OrdinalEncoder()` của `sklearn` trong python. Tôi thực hiện thử nghiệm sau đây để mã hoá biến định tính dạng String.

Chương trình này sẽ chuyển tất các các biến dạng string có từ 2 -> dưới 8 lựa chọn thành dạng biến dummy (biến danh mục - không phân theo thứ hạng) và biến theo thứ hạng. Việc xác định biến danh mục với thứ hạng là bước đầu tiên

1. Tập danh sách các biến string

2. Đòng bộ biến string đó (bỏ khoảng trống thừa, chuyển về lowercase, ... )

3. Phân loại biến danh mục và thứ hạng

4. Mã hoá biến danh mục thành dummy

5. Mã hoá biến thứ hạng

In [ ]:
// 1. Lập danh sách các biến string
qui ds, has(type string)
local strvar `r(varlist)'

// 2. Đồng bộ biến string (bỏ khoảng trống thừa, chuyển về lowercase)
foreach vr of local strvar {
    qui replace `vr' = ustrlower(`vr')
    qui replace `vr' = ustrtrim(`vr')
}

// 3. Lập danh sách các biến mục tiêu để chuyển thành biến dummy
local canvar ""
la de yesno 0 "Không" 1 "Có", modify
set varabbrev off

foreach vr of local strvar {
    qui tab `vr'
    if `r(r)' >=2 & `r(r)'<=8 local canvar = "`canvar'" + " `vr'"
}
if strlen("`canvar'") == 0 {
    di in re "Không có biến string nào có từ 2-8 lựa chọn"
    error 102
}


// 4. Chuyển biến số về dạng dummy
foreach vr of local canvar {
    qui tab `vr', gen(`vr')
    local nval `r(r)'

    * tạo nhãn biến mới 
    forvalues i=1/`nval' {
        local varlabel`i' : var lab `vr'
        local varlabel`i' = subinstr("`varlabel`i''", "`vr'==", "", .)
    }


    * Gán nhãn cho biến số
    forvalues i=1/`nval' {
        la var `vr'`i' "`varlabel`i''"
        la val `vr'`i' yesno
    }
}

* Xoá những biến số cũ
qui drop `canvar'

In [ ]:
* Chương trình mã hoá string dưới dạng biến thứ hạng
cap program drop encodeordinal
program define encodeordinal
    syntax varlist
    foreach vr of varlist `varlist' {
        qui replace `vr' = ustrtrim(`vr')
        qui encode `vr', gen(`vr'_en) label(label_`vr')
        order `vr'_en, after(`vr')
    }    
    qui drop `varlist'
end

## Các bước thực hiện làm sạch:

1. Xác định biến thứ hạng

2. Chạy chương trình mã hoá string dưới dạng biến thứ hạng

3. Chạy mã hoá biến string còn lại (từ 2-8 lựa chọn) thành biến dummy